# Setup

In [2]:
include(joinpath("..","src","setup.jl"))

datadir_all = joinpath("..","data","count_lengths","run_2018-11-28")
datadir_selected = joinpath("..","data","count_lengths","run_2018-11-30")


┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840
┌ Info: Loading Cairo backend into Compose.jl
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/Compose.jl:164
┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/mapping.jl:228


"../data/count_lengths/run_2018-11-30"

# Load and Summarize Data

In [16]:
params_all = load_params(joinpath(datadir_all,"params.jld2"))
params_all[:pindex] = 1:size(params_all,1)
settings = joinpath(srcdir,"settings.toml")

results = []
for_results_in(joinpath(datadir_all,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df_all = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:31


In [21]:
params_selected = load_params(joinpath(datadir_selected,"params.jld2"))
params_selected[:pindex] = 1:size(params_selected,1)
settings = joinpath(srcdir,"settings.toml")

results = []
for_results_in(joinpath(datadir_selected,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df_selected = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09


In [24]:
herr = human_error(N=N_for_pressnitzer_hupe_2006)

(stream = 0.1251241594611916, lengths = 0.21709315068493154)

In [26]:
fields = [:f_c_a,:f_c_m,:f_c_σ,:s_c_a,:s_c_m,:s_c_σ,:t_c_a,:t_c_m,:t_c_σ]
progress = Progress(nrow(unique(params_all[fields])))
df_summary_all = by(@where(params_all,:Δf .== 6),fields) do row
    next!(progress)
    
    err =
        model_error(df_all,params_all,
                  f_c_a=row[1,:f_c_a], f_c_m=row[1,:f_c_m], f_c_σ = row[1,:f_c_σ],
                  s_c_a=row[1,:s_c_a], s_c_m=row[1,:s_c_m], s_c_σ = row[1,:s_c_σ],
                  t_c_a=row[1,:t_c_a], t_c_m=row[1,:t_c_m], t_c_σ = row[1,:t_c_σ])
    eratio = error_ratio(err,herr)
    DataFrame([merge((eratio=eratio,),(pindex = row.pindex[1],))])
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:08


,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,eratio,pindex
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,2.28641,1
2,0.0,5.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,1.99579,2
3,0.0,63.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,3.32,3
4,0.0,790.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,3.32,4
5,0.0,10000.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,3.32,5
6,0.0,0.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,2.17073,6
7,0.0,5.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,2.16691,7
8,0.0,63.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,3.32,8
9,0.0,790.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,3.32,9


In [27]:
fields = [:f_c_a,:f_c_m,:f_c_σ,:s_c_a,:s_c_m,:s_c_σ,:t_c_a,:t_c_m,:t_c_σ]
progress = Progress(nrow(unique(params_selected[fields])))
df_summary_selected = by(@where(params_selected,:Δf .== 6),fields) do row
    next!(progress)
    
    err =
        model_error(df_selected,params_selected,
                  f_c_a=row[1,:f_c_a], f_c_m=row[1,:f_c_m], f_c_σ = row[1,:f_c_σ],
                  s_c_a=row[1,:s_c_a], s_c_m=row[1,:s_c_m], s_c_σ = row[1,:s_c_σ],
                  t_c_a=row[1,:t_c_a], t_c_m=row[1,:t_c_m], t_c_σ = row[1,:t_c_σ])
    eratio = error_ratio(err,herr)
    DataFrame([merge((eratio=eratio,),(pindex = row.pindex[1],))])
end

Progress:  87%|████████████████████████████████████     |  ETA: 0:00:01┌ Warning: Expected 3 parameter entries. 
│ Instead, only found entires: 2×36 DataFrame
│ │ Row │ f_W_m_σ │ f_c_m   │ t_c_a   │ t_c_m   │ f         │ t_W_m_c │ t_c_x   │ f_c_x   │ t_W_m_σ_t │ s_τ_σ     │ f_c_a   │ f_c_σ   │ s_c_m   │ f_τ_a     │ s_c_σ   │ Δt        │ s_τ_m     │ s_c_x   │ t_W_m_σ_ϕ │ t_τ_a     │ s_c_a   │ t_τ_m     │ t_c_σ   │ t_τ_x     │ s_W_m_c │ s_τ_a     │ f_W_m_c │ t_τ_σ     │ f_τ_σ     │ f_τ_x     │ Δf    │ t_W_m_σ_N │ f_τ_m     │ s_τ_x     │ s_W_m_σ │ pindex │
│ │     │ Float64 │ Float64 │ Float64 │ Float64 │ Quantity… │ Float64 │ Float64 │ Float64 │ Float64   │ Quantity… │ Float64 │ Float64 │ Float64 │ Quantity… │ Float64 │ Quantity… │ Quantity… │ Float64 │ Float64   │ Quantity… │ Float64 │ Quantity… │ Float64 │ Quantity… │ Float64 │ Quantity… │ Float64 │ Quantity… │ Quantity… │ Quantity… │ Int64 │ Float64   │ Quantity… │ Quantity… │ Float64 │ Int64  │
│ ├─────┼─────────┼─────────┼─────────┼

,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,eratio,pindex
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,5.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,1.16955,1405
2,5.0,5.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,2.33323,1406
3,5.0,63.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.964238,1407
4,5.0,0.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,1.30681,1408
5,5.0,5.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,2.43407,1409
6,5.0,63.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,1.00181,1410
7,5.0,0.0,0.2,0.0,0.0,0.2,63.0,0.0,0.2,1.18171,1411
8,5.0,5.0,0.2,0.0,0.0,0.2,63.0,0.0,0.2,2.33885,1412
9,5.0,63.0,0.2,0.0,0.0,0.2,63.0,0.0,0.2,0.985413,1413


In [8]:
# if there is a selected entry with a revised rms, use that value

df_summary_selected[:procedure] = "selected"
df_summary_all[:procedure] = "survey"
function favor_selected(vals,procedure)
    selected = findfirst(procedure .== "selected")
    if selected isa Nothing
        @assert length(vals) == 1
        first(vals)
    else
        vals[selected]
    end
end
    
df_summary = @linq vcat(df_summary_all,df_summary_selected) |>
    by(fields,rms = favor_selected(:rms,:procedure))

UndefVarError: UndefVarError: df_summary_selected not defined

# Plots by Level

In [9]:
mlevels = [(vars = [:f_c_a,:f_c_m], name = "Peripheral"),
          (vars = [:s_c_a,:s_c_m], name = "Cortical"),
          (vars = [:t_c_a,:t_c_m], name = "Object")]
dflevels = map(mlevels) do level
    result = by(df_summary,level.vars) do slice
       DataFrame(c_a = first(slice[level.vars[1]]),c_m = first(slice[level.vars[2]]),
                 level = level.name, rms = minimum(skipmissing(slice.rms)))
    end
    result[[:c_a,:c_m,:level,:rms]]
end
dfplot = @linq vcat(dflevels...) |>
    transform(a = packaxes(:c_a,maxpad=false), m = packaxes(:c_m,maxpad=false)) |>
    transform(rms_str = fmt.("01.2f",:rms))
head(dfplot)

UndefVarError: UndefVarError: df_summary not defined

In [10]:
scale = reverse!(colormap("RdBu",mid=(1.5-0.5)/1.5));
function bluered(;minvalue,maxvalue,colorstart=minvalue,colorstop=maxvalue,midvalue=(colorstop-colorstart)/2+colorstart)
    scale(x,min,max) = (x - min)/(max - min)
    scalei(x,min,max) = x*(max - min) + min
    fi(x) = scalei(clamp(x,minvalue,maxvalue),minvalue,maxvalue)
    c(x) = scale(x,colorstart,colorstop)
    colors = reverse!(colormap("RdBu",mid=1-c(midvalue)))
    
    function(x)
        colors[1+clamp(floor(Int,99*c(fi(x))),0,99)]
    end
end

bluered (generic function with 1 method)

In [11]:
xvals = sort!(unique(dfplot.a));
yvals = sort!(unique(dfplot.m));

UndefVarError: UndefVarError: dfplot not defined

In [12]:
pl = plot(dfplot,x=:a,y=:m,color=:rms,xgroup=:level,label=:rms_str,
     Geom.subplot_grid(Geom.label(position=:centered),Geom.rectbin,Coord.cartesian(ymin=0,xmin=0),
                       Scale.x_continuous(labels=string ∘ Int ∘ packaxes_invfn(dfplot.c_a,maxpad=false)),
                       Scale.y_continuous(labels=string ∘ Int ∘ packaxes_invfn(dfplot.c_m,maxpad=false)),
                       Guide.xticks(ticks=xvals,orientation=:vertical),
                       Guide.yticks(ticks=yvals,orientation=:horizontal)),
     Guide.xlabel("Adaptation"), Guide.ylabel("Inhibition"),
     Scale.color_continuous(minvalue=0,maxvalue=2,colormap=bluered(minvalue=0,maxvalue=2,colorstart=0.5,colorstop=3,midvalue=1.0)),
     Guide.colorkey(title="Mean Relative Error"),
     Theme(grid_line_width=0inch,point_label_font_size=4pt,point_label_color=colorant"black"))
draw(PDF(joinpath(grantdir,"fig3C.pdf"),9inch,3.5inch),pl)
draw(PNG(12inch,4inch),pl)

UndefVarError: UndefVarError: dfplot not defined

# Select parameters for further simulations

Before we have run df_summary_select, we select parameters from df_summary_all that have an rms ratio < 1

In [41]:
df_selected = @where(df_summary_all,:eratio .< 1);
head(df_selected)

,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,eratio,pindex
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,5.0,63.0,0.2,0.0,0.0,0.2,0.0,0.0,0.2,0.752069,128
2,5.0,63.0,0.2,0.0,0.0,0.2,5.0,0.0,0.2,0.785466,133
3,5.0,63.0,0.2,0.0,0.0,0.2,63.0,0.0,0.2,0.868803,138
4,5.0,63.0,0.2,0.0,0.0,0.2,790.0,0.0,0.2,0.868812,143
5,5.0,63.0,0.2,0.0,0.0,0.2,10000.0,0.0,0.2,0.856736,148
6,5.0,0.0,0.2,0.0,0.0,0.2,0.0,5.0,0.2,0.882262,151


In [42]:
println("Selected ",string(round(100size(df_selected,1) / size(df_summary_all,1),digits=2)),"%")

Selected 4.22%


In [37]:
size(df_selected,1)

659

In [26]:
pindices = unique(df_selected[:pindex]);

In [27]:
new_params = params[pindices,:];
df12 = deepcopy(new_params)
df12.Δf = 12
df3 = deepcopy(new_params)
df3.Δf = 3
new_params = vcat(df3,new_params,df12);

In [28]:
new_datadir = joinpath("..","data","count_lengths","run_$(Date(now()))")
isdir(new_datadir) || mkdir(new_datadir)
save(joinpath(new_datadir,"params.jld2"),"params",new_params)

In [29]:
open(joinpath(new_datadir,"interact_N.txt"),"w") do f
    println(f,"$(size(new_params,1))")
end
